In [131]:
# @title Upload CSV
import pandas as pd

hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')

In [132]:
# @title Join MK dan Dosen
# Gabungkan data_dosen dengan dosen
merged_df = pd.merge(data_dosen_df, dosen_df, on='id_dosen', how='inner')

# Gabungkan hasilnya dengan mk_genap
merged_df = pd.merge(merged_df, mk_genap_df, on='id_mk_genap', how='inner')

# Tampilkan hasil gabungan
print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    
    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [133]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
  wolf = []
  for id_counter, (hari, ruang, jam) in enumerate(
      [(h, r, j) for h in hari_list for r in ruang_list for j in jam_list], start=1
  ):
      wolf.append({
          "id_slot": id_counter,
          "mata_kuliah": None,
          "dosen": None,
          "ruang": ruang,
          "hari": hari,
          "jam_mulai": jam['jam_awal'],
          "jam_selesai": jam['jam_akhir'],
          "kelas": None,
          "sks": None,
          "metode": None
      })
  return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [140]:
# @title Inisialisasi Populasi (GWO)
import random

def create_random_schedule():
    slots = slot_generator()
    merged_shuffled = merged_df.sample(frac=1).iterrows()

    for index, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']

        possible_positions = list(range(len(slots) - sks + 1))
        random.shuffle(possible_positions)

        for i in possible_positions:
            block = slots[i:i+sks]
            
            # Cek kelayakan slot
            all_empty = all(slot['mata_kuliah'] is None for slot in block)
            same_hari = len(set(slot['hari'] for slot in block)) == 1
            same_ruang = (metode == 'Online') or (len(set(slot['ruang'] for slot in block)) == 1)
            
            if all_empty and same_ruang or same_hari:
                # Cek apakah dosen sudah ada jadwal di hari tersebut
                target_hari = block[0]['hari']
                dosen_occupied = any(
                    slot['dosen'] == dosen and slot['hari'] == target_hari
                    for slot in slots
                    if slot['mata_kuliah'] is not None
                )
                
                if not dosen_occupied:
                    ruang = 'Online' if metode == 'Online' else block[0]['ruang']
                    
                    # Update semua slot dalam blok
                    for slot in block:
                        slot.update({
                            "mata_kuliah": mata_kuliah,
                            "dosen": dosen,
                            "kelas": kelas,
                            "sks": sks,
                            "metode": metode,
                            "ruang": ruang
                        })
                    break
    return slots

population_size = 3
population = [create_random_schedule() for _ in range(population_size)]

for i in range(population_size):
    print(f"\nIndividu {i+1}:")
    for slot in population[i][:5]:
        print(slot)


Individu 1:
{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}

Individu 2:
{'id_slot': 1, 'mata_kuliah': 'Arsitektur Komputer', 'do

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
jadwal dosen mengajar kelas yang berbeda pada jam yang sama
jadwal ruang kelas ruang kelas digunakan lebih dari 1 dosen 

Konstrain ringan:
dosen mengajar pada hari yang sama
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [146]:
from collections import defaultdict
from datetime import datetime

def calculate_fitness(schedule):
    max_score = 100
    penalty = 0
    violation_details = {
        'heavy': defaultdict(list),
        'light': defaultdict(list)
    }

    # Konversi waktu ke menit
    def time_to_minutes(t):
        try:
            return datetime.strptime(t, "%H:%M:%S").hour * 60 + datetime.strptime(t, "%H:%M:%S").minute
        except ValueError:
            return datetime.strptime(t, "%H:%M").hour * 60 + datetime.strptime(t, "%H:%M").minute

    # Struktur tracking
    teacher_schedule = defaultdict(lambda: defaultdict(list))
    classroom_schedule = defaultdict(lambda: defaultdict(list))
    teacher_days = defaultdict(set)
    day_time_bounds = defaultdict(lambda: {'min': float('inf'), 'max': -1})

    # Kumpulkan data
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
            
        hari = slot['hari']
        dosen = slot['dosen']
        ruang = slot['ruang']
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        # Track untuk heavy constraints
        teacher_schedule[dosen][hari].append((start, end))
        classroom_schedule[ruang][hari].append((start, end))
        
        # Track untuk light constraints
        teacher_days[dosen].add(hari)
        
        # Update batas waktu per hari
        day_time_bounds[hari]['min'] = min(day_time_bounds[hari]['min'], start)
        day_time_bounds[hari]['max'] = max(day_time_bounds[hari]['max'], end)

    # 1. Heavy Constraints (1 point per violation)
    # a. Dosen mengajar kelas berbeda di waktu sama
    for dosen, days in teacher_schedule.items():
        for hari, times in days.items():
            sorted_times = sorted(times, key=lambda x: x[0])
            for i in range(1, len(sorted_times)):
                if sorted_times[i][0] < sorted_times[i-1][1]:
                    penalty += 1
                    violation_details['heavy']['dosen'].append(
                        f"{dosen} di {hari} {convert_time(sorted_times[i-1])} vs {convert_time(sorted_times[i])}"
                    )
    
    # b. Ruang digunakan lebih dari 1 kelas di waktu sama
    for ruang, days in classroom_schedule.items():
        for hari, times in days.items():
            sorted_times = sorted(times, key=lambda x: x[0])
            for i in range(1, len(sorted_times)):
                if sorted_times[i][0] < sorted_times[i-1][1]:
                    penalty += 1
                    violation_details['heavy']['ruang'].append(
                        f"{ruang} di {hari} {convert_time(sorted_times[i-1])} vs {convert_time(sorted_times[i])}"
                    )

    # 2. Light Constraints (0.5 point per violation)
    # a. Dosen mengajar di hari yang sama (meski waktu berbeda)
    for dosen, days in teacher_days.items():
        if len(days) < 2:
            continue
        penalty += 0.5 * len(days)
        violation_details['light']['dosen_hari'].append(
            f"{dosen} mengajar di {len(days)} hari berbeda"
        )
    
    # b. Kelas di waktu paling awal/akhir
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
            
        hari = slot['hari']
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        if start == day_time_bounds[hari]['min']:
            penalty += 0.5
            violation_details['light']['waktu_ekstrim'].append(
                f"Kelas {slot['kelas']} di {hari} jam pertama ({slot['jam_mulai']})"
            )
            
        if end == day_time_bounds[hari]['max']:
            penalty += 0.5
            violation_details['light']['waktu_ekstrim'].append(
                f"Kelas {slot['kelas']} di {hari} jam terakhir ({slot['jam_selesai']})"
            )

    fitness = max(max_score - penalty, 0)
    
    return {
        'fitness': round(fitness, 2),
        'penalty': round(penalty, 2),
        'violations': dict(violation_details)
    }

def convert_time(time_tuple):
    def to_hhmm(minutes):
        return f"{minutes//60:02}:{minutes%60:02}"
    return f"{to_hhmm(time_tuple[0])}-{to_hhmm(time_tuple[1])}"

population = [create_random_schedule() for _ in range(10)]

for idx, schedule in enumerate(population):
    result = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}")
    print(f"Fitness: {result['fitness']}")
    print(f"Penalty: {result['penalty']}")
    print("Detil Pelanggaran:")
    
    print("\nKonstrain Berat:")
    for category, violations in result['violations']['heavy'].items():
        print(f"- {category}:")
        for v in violations[:2]:  # Tampilkan 2 contoh
            print(f"  • {v}")
    
    print("\nKonstrain Ringan:")
    for category, violations in result['violations']['light'].items():
        print(f"- {category}:")
        for v in violations[:2]:
            print(f"  • {v}")


Individu 1
Fitness: 3.0
Penalty: 97.0
Detil Pelanggaran:

Konstrain Berat:
- ruang:
  • Online di Senin 07:00-07:50 vs 07:00-07:50
  • Online di Senin 07:00-07:50 vs 07:00-07:50

Konstrain Ringan:
- dosen_hari:
  • Sheraton Pawestri, S.Kom., M.Cs. mengajar di 2 hari berbeda
  • Ardiansyah, Dr., S.T., M.Cs. mengajar di 4 hari berbeda
- waktu_ekstrim:
  • Kelas B di Senin jam terakhir (17:50:00)
  • Kelas B di Senin jam pertama (7:00:00)

Individu 2
Fitness: 1.5
Penalty: 98.5
Detil Pelanggaran:

Konstrain Berat:
- ruang:
  • Online di Senin 09:35-10:25 vs 09:35-10:25
  • Online di Senin 10:30-11:20 vs 10:30-11:20

Konstrain Ringan:
- dosen_hari:
  • Anna Hendri Soleliza Jones, S.Kom., M.Cs. mengajar di 2 hari berbeda
  • Murein Miksa Mardhia, S.T., M.T. mengajar di 4 hari berbeda
- waktu_ekstrim:
  • Kelas D di Senin jam terakhir (17:50:00)
  • Kelas D di Senin jam pertama (7:00:00)

Individu 3
Fitness: 0.0
Penalty: 100.0
Detil Pelanggaran:

Konstrain Berat:
- ruang:
  • 4.1.5.60 di Sel

In [136]:
# @title Postprocessing Return solusi terbaik (Alpha).
max_iter = 10

def get_best_wolves(population):
    fitness_scores = [(schedule, calculate_fitness(schedule)) for schedule in population]
    sorted_wolves = sorted(fitness_scores, key=lambda x: x[1], reverse=True)
    return sorted_wolves[0][0], sorted_wolves[1][0], sorted_wolves[2][0]

for iterasi in range(max_iter):
    alpha, beta, delta = get_best_wolves(population)
    new_population = []

    for wolf in population:
        if wolf in [alpha, beta, delta]:
            new_population.append(wolf)
            continue
        new_wolf = copy.deepcopy(wolf)
        for slot in new_wolf:
            alpha_slot = next((s for s in alpha if s["id_slot"] == slot["id_slot"]), None)
            beta_slot = next((s for s in beta if s["id_slot"] == slot["id_slot"]), None)
            delta_slot = next((s for s in delta if s["id_slot"] == slot["id_slot"]), None)
            if alpha_slot and beta_slot and delta_slot:
                choices = [alpha_slot, beta_slot, delta_slot]
                best_choice = sorted(choices, key=lambda x: x["id_slot"])[0]
                slot.update({"mata_kuliah": best_choice["mata_kuliah"], "dosen": best_choice["dosen"], "kelas": best_choice["kelas"]})
        new_population.append(new_wolf)
    population = new_population
    print(f"Iterasi {iterasi+1} - Best Fitness: {calculate_fitness(alpha)}")

final_schedule = get_best_wolves(population)[0]
for slot in sorted(final_schedule, key=lambda x: x["id_slot"]):
    print(slot)

TypeError: '<' not supported between instances of 'dict' and 'dict'